In [3]:
import cv2
import numpy as np

def add_cracks(image, num_cracks=5, crack_width=2):
    result = image.copy()
    height, width = image.shape[:2]
    for _ in range(num_cracks):
        start_point = (np.random.randint(0, width), np.random.randint(0, height))
        end_point = (np.random.randint(0, width), np.random.randint(0, height))
        color = (np.random.randint(0, 50), np.random.randint(0, 50), np.random.randint(0, 50))
        cv2.line(result, start_point, end_point, color, crack_width)
    return result

def add_color_fading(image, fade_factor=0.7):
    return cv2.addWeighted(image, fade_factor, np.ones_like(image) * 255, 1 - fade_factor, 0)

def add_scratches(image, num_scratches=10, max_length=50):
    result = image.copy()
    height, width = image.shape[:2]
    for _ in range(num_scratches):
        start_point = (np.random.randint(0, width), np.random.randint(0, height))
        angle = np.random.uniform(0, 2 * np.pi)
        length = np.random.randint(10, max_length)
        end_point = (
            int(start_point[0] + length * np.cos(angle)),
            int(start_point[1] + length * np.sin(angle))
        )
        cv2.line(result, start_point, end_point, (200, 200, 200), 1)
    return result

def add_water_damage(image, num_spots=5, max_radius=50):
    result = image.copy()
    height, width = image.shape[:2]
    for _ in range(num_spots):
        center = (np.random.randint(0, width), np.random.randint(0, height))
        radius = np.random.randint(10, max_radius)
        color = (np.random.randint(150, 200), np.random.randint(150, 200), np.random.randint(150, 200))
        cv2.circle(result, center, radius, color, -1)
        result = cv2.addWeighted(result, 0.7, image, 0.3, 0)
    return result

def add_mold(image, num_spots=10, max_radius=20):
    result = image.copy()
    height, width = image.shape[:2]
    for _ in range(num_spots):
        center = (np.random.randint(0, width), np.random.randint(0, height))
        radius = np.random.randint(5, max_radius)
        color = (np.random.randint(0, 100), np.random.randint(50, 150), np.random.randint(0, 100))
        cv2.circle(result, center, radius, color, -1)
    return result

def add_noise(image, mean=0, std=25):
    noise = np.random.normal(mean, std, image.shape).astype(np.uint8)
    return cv2.add(image, noise)

def apply_damage_effects(image):
    damaged_image = image.copy()

    # 랜덤하게 효과 선택 및 적용
    if np.random.rand() < 0.3:
        damaged_image = add_cracks(damaged_image, num_cracks=np.random.randint(3, 8))
    if np.random.rand() < 0.4:
        damaged_image = add_color_fading(damaged_image, fade_factor=np.random.uniform(0.6, 0.9))
    if np.random.rand() < 0.6:
        damaged_image = add_scratches(damaged_image, num_scratches=np.random.randint(5, 15))
    if np.random.rand() < 0.3:
        damaged_image = add_water_damage(damaged_image, num_spots=np.random.randint(2, 6))
    if np.random.rand() < 0.5:
        damaged_image = add_mold(damaged_image, num_spots=np.random.randint(5, 15))
    if np.random.rand() < 0.4:
        damaged_image = add_noise(damaged_image, std=np.random.randint(10, 30))

    return damaged_image

# 입력 및 출력 경로 직접 지정
input_path = "./images/test1.jpg"
output_path = "./images/test2.jpg"

# 이미지 읽기
image = cv2.imread(input_path)

if image is None:
    print(f"Failed to read image: {input_path}")
else:
    # 손상 효과 적용
    damaged_image = apply_damage_effects(image)

    # 손상된 이미지 저장
    cv2.imwrite(output_path, damaged_image)
    print(f"Damaged image saved to: {output_path}")

Damaged image saved to: ./images/test2.jpg


In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from time import time

def detect_and_match_features(img1, img2, detector, matcher):
    start = time()
    kp1, des1 = detector.detectAndCompute(img1, None)
    kp2, des2 = detector.detectAndCompute(img2, None)
    end = time()
    detection_time = end - start

    start = time()
    if des1 is not None and des2 is not None:
        matches = matcher.match(des1, des2)
        matches = sorted(matches, key=lambda x: x.distance)
    else:
        matches = []
    end = time()
    matching_time = end - start

    return kp1, kp2, matches, detection_time, matching_time

def draw_matches(img1, kp1, img2, kp2, matches, max_matches=50):
    matches = matches[:max_matches]
    return cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# 이미지 로드
img1 = cv2.imread('./images/test1.jpg')
img2 = cv2.imread('./images/test2.jpg')

# 그레이스케일로 변환
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# 특징점 검출기 및 매칭기 초기화
sift = cv2.SIFT_create()
orb = cv2.ORB_create()
akaze = cv2.AKAZE_create()

bf_matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
bf_matcher_hamming = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# 각 알고리즘에 대해 특징점 검출 및 매칭 수행
algorithms = [
    ('SIFT', sift, bf_matcher),
    ('ORB', orb, bf_matcher_hamming),
    ('AKAZE', akaze, bf_matcher_hamming)
]

results = {}
for name, detector, matcher in algorithms:
    kp1, kp2, matches, detection_time, matching_time = detect_and_match_features(gray1, gray2, detector, matcher)
    results[name] = {
        'kp1': kp1,
        'kp2': kp2,
        'matches': matches,
        'detection_time': detection_time,
        'matching_time': matching_time
    }

# 결과 시각화
plt.figure(figsize=(15, 5))

for i, (name, result) in enumerate(results.items()):
    plt.subplot(1, 3, i+1)
    matched_img = draw_matches(img1, result['kp1'], img2, result['kp2'], result['matches'])
    plt.imshow(cv2.cvtColor(matched_img, cv2.COLOR_BGR2RGB))
    plt.title(f'{name}: {len(result["matches"])} matches')
    plt.axis('off')

plt.tight_layout()
plt.savefig('./images/feature_matching_comparison.png')
plt.close()

# 성능 비교 그래프
detection_times = [result['detection_time'] for result in results.values()]
matching_times = [result['matching_time'] for result in results.values()]
num_matches = [len(result['matches']) for result in results.values()]

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

ax1.bar(results.keys(), detection_times)
ax1.set_title('Detection Time')
ax1.set_ylabel('Time (seconds)')

ax2.bar(results.keys(), matching_times)
ax2.set_title('Matching Time')
ax2.set_ylabel('Time (seconds)')

ax3.bar(results.keys(), num_matches)
ax3.set_title('Number of Matches')
ax3.set_ylabel('Matches')

plt.tight_layout()
plt.savefig('./images/performance_comparison.png')
plt.close()

# 결과 출력
for name, result in results.items():
    print(f"{name}:")
    print(f"  Number of keypoints in original image: {len(result['kp1'])}")
    print(f"  Number of keypoints in damaged image: {len(result['kp2'])}")
    print(f"  Number of matches: {len(result['matches'])}")
    print(f"  Detection time: {result['detection_time']:.4f} seconds")
    print(f"  Matching time: {result['matching_time']:.4f} seconds")
    print()

print("Results saved as 'feature_matching_comparison.png' and 'performance_comparison.png'")

AttributeError: module 'cv2' has no attribute 'xfeatures2d'